# Pipelines

In [ ]:
%run "./Getting started.ipynb"
import requests
base_url = 'http://localhost:6000'
while True:
    try:
        r=requests.get(base_url)
        if r.text == 'EMERGENT API':
            break
    except:
        continue
print(r.text)

The Pipeline class allows us to combine multiple optimization processes into a single process. For example, we could do a coarse grid search to obtain an initial signal, then shrink the bounds to enclose the signal and sample with higher resolution, then finally fit all observed data to a model which predicts the location of the maximum:

In [ ]:
import numpy as np
from emergent.pipeline import (Pipeline, Source, GridSearch, GaussianModel, Rescale, DifferentialEvolution, 
                              ParticleSwarm, GaussianProcess, GradientDescent, Adam, LBFGSB, Prune)
from emergent.utilities.containers import DataDict

''' Define data source '''
hub = network.hubs['hub']
hub.range = DataDict({'thing': {'X': {'min': -2, 'max': 2}, 'Y': {'min': -2, 'max': 2}}})
thing = hub.children['thing']
state = {'thing': {'X': 0.1, 'Y': .9}}
bounds = hub.range.copy()
experiment = hub.gaussian
params = {'sigma_x': 0.3, 'sigma_y': 0.8, 'x0': 0.3, 'y0': 0.6, 'noise':0.0}
source = Source(state, bounds, experiment, params)

''' Declare and run pipeline.  Allowed blocks are:
    Sampling methods: GridSearch, DifferentialEvolution, ParticleSwarm
    Models: GaussianModel
    Other: Rescale '''
pipe = Pipeline(state, source)

# pipe.add(LBFGSB())
# pipe.add(GridSearch(params={'Steps': 10}))
# pipe.add(Rescale(threshold=0.75))

pipe.add(ParticleSwarm({'Inertia': 0.3, 'Cognitive acceleration': 1, 'Social acceleration': 1}))
# pipe.add(DifferentialEvolution())

# pipe.add(GridSearch(params={'Steps':10}))
pipe.add(GaussianModel(optimizer = DifferentialEvolution()))




points, costs = pipe.run()
pipe.plot()
print(points[-1], costs[-1])


In [ ]:
model = pipe.blocks[1]
model.plot(0,1, n_points=40)


In [ ]:
self=pipe
import matplotlib.pyplot as plt
import seaborn
# seaborn.set()
plt.plot(self.costs, '.k')
plt.plot(np.minimum.accumulate(self.costs), '--k')
plt.show()



In [ ]:
x = np.linspace(0,1,100)
y = np.linspace(0,1,100)
predict_points = np.transpose(np.meshgrid(x,y)).reshape(-1, 2)
len(predict_points)